#### Which burrito trucks to select?

#### 0. Importing libraries

In [1]:
%pip install gurobipy
import gurobipy as gp
from gurobipy import GRB
import numpy as np

Note: you may need to restart the kernel to use updated packages.


#### 1. Set up the model

#### 2. Define sets and parameters

#### 3. Set the variables and the objective function

#### 4. Set the "sense" of the optimization problem

#### 5. Add the constraints

#### 6. Update and optimize the model

#### 7. Print the results